In [4]:
import nltk
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
import string
import pandas as pd
from string import punctuation
punctuation = set(punctuation)
from nltk.stem import PorterStemmer



def process_text(text):
    '''Function to process text fields.
    Involves removing punctuation, tokenizing text, removing stopwords, lemmatizing tokens, folding to lowercase.'''
    # Define punctuation set
    punctuation = set(string.punctuation)
    # Add additional punctuation character
    punctuation.update({'‘'})

    # Tokenize text using NLTK's word_tokenize
    tokens = word_tokenize(text)
    
    # Remove stopwords and punctuation, and lemmatize tokens
    sw = set(stopwords.words("english"))
    lemmatizer = WordNetLemmatizer()
    tokens = [
        lemmatizer.lemmatize(token.lower())
        for token in tokens
        if token.lower() not in sw and all(char not in punctuation for char in token)
    ]
    
    return tokens

In [5]:
# Define the text processing function
def process_label_text(text):
    if isinstance(text, str):  # Check if the input is a string
        # Define punctuation set
        punctuation = set(string.punctuation)
        
        # Replace punctuation with spaces
        for p in punctuation:
            text = text.replace(p, ' ')
        
        # Tokenize text using NLTK's word_tokenize
        tokens = word_tokenize(text)

        # Remove stopwords and lemmatize tokens
        sw = set(stopwords.words("english"))
        lemmatizer = WordNetLemmatizer()
        tokens = [
            lemmatizer.lemmatize(token.lower())
            for token in tokens
            if token.lower() not in sw
        ]

        # Remove duplicate tokens while maintaining order
        seen = set()
        unique_tokens = []
        for token in tokens:
            if token not in seen:
                seen.add(token)
                unique_tokens.append(token)
        
        return unique_tokens  # Return the list of tokens
    else:
        return text  # Return the original value if it's not a string


In [6]:
def add_sequential_index(df, index_col_name):
    """
    Add sequential index column to dataframe table. Inputs are the dataframe, and desired name of index column. 
    """
    # Reset the index and rename the index column to input index_col_name
    df = df.reset_index().rename(columns={"index": index_col_name})
    
    # Add 1 to index to start index from 1 instead of 0
    df[index_col_name] = df[index_col_name] + 1
    
    return df

In [3]:
# Missing values to null for now (simplifies type conversions & plotting)
def na_to_null(df, column):
    df[column] = df[column].replace('N/A', np.nan)
    return df